# Simatree Assessment: Data Cleaning

Welcome to the data cleaning portion of the Simatree Assessment. This notebook will provide a series of code chunks to import, format, clean, and export the provided Athlete Events data. This notebook will follow the steps below:
1. Begin by reading the CSV file to a Pandas data frame.
2. Impute missing Age, Height, and Weight values using group averages.
3. Format the Medals column.
4. Format the Name column.
5. Remove duplicate values within the data frame.
6. Export the data frame to a CSV file.

In [1]:
import pandas as pd
import os
import csv
import math

import re
import datetime
import io
from zipfile import ZipFile

import pandas as pd
import numpy as np
import pandas_gbq
import janitor
from sklearn.impute import KNNImputer
from sklearn.preprocessing import LabelEncoder

## Read Athlete data into Pandas Dataframe

In [2]:
new_df = pd.read_csv('athlete_events.csv')

## Data Cleaning 

### Impute missing Age and Weight values

I'll begin by imputing Age, Height, and Weight NA values by grouping athletes by Sex and Sport. I'll then use the fillna method to fill in missing height and weight values with the mean height and weight value within each grouping.

In [3]:
#create groups from the athlete data with the same Sex and Sport values
grouped = new_df.groupby(['Sex', 'Sport'])

# Create a function that imputes missing values with the mean Age of the group, using the fillna method
def impute_age(group):
    group['Age'] = group['Age'].fillna(group['Age'].mean())
    return group

# Apply the function to each group and assign the results to new_df
new_df = grouped.apply(impute_age).reset_index(drop=True)

In [4]:
#create groups from the athlete data with the same Sex and Sport values
grouped = new_df.groupby(['Sex', 'Sport'])

# Create a function that imputes missing values with the mean Height of the group, using the fillna method
def impute_height(group):
    group['Height'] = group['Height'].fillna(group['Height'].mean())
    return group

# Apply the function to each group and assign the results to new_df
new_df = grouped.apply(impute_height).reset_index(drop=True)


In [5]:
#create groups from the athlete data with the same Sex and Sport values
grouped = new_df.groupby(['Sex', 'Sport'])

# Create a function that imputes missing values with the mean Weight of the group, using the fillna method
def impute_weight(group):
    group['Weight'] = group['Weight'].fillna(group['Weight'].mean())
    return group

# Apply the function to each group and assign the results to new_df
new_df = grouped.apply(impute_weight).reset_index(drop=True)

I'll end the imputation by creating a new set of groups based on Sex and National Olympic Committee (country) as there are still 100 missing Height and Weight values after the imputation above.

In [6]:
#create groups from the athlete data with the same Sex and Sport values
grouped = new_df.groupby(['Sex', 'NOC'])

# Create a function that imputes missing values with the mean Age of the group, using the fillna method
def impute_age(group):
    group['Age'] = group['Age'].fillna(group['Age'].mean())
    return group

# Apply the function to each group and assign the results to new_df
new_df = grouped.apply(impute_age).reset_index(drop=True)

In [7]:
#create groups from the athlete data with the same Sex and NOC values
grouped = new_df.groupby(['Sex', 'NOC'])

# Create a function that imputes missing values with the mean Height of the group, using the fillna method
def impute_height(group):
    group['Height'] = group['Height'].fillna(group['Height'].mean())
    return group

# Apply the function to each group and assign the results to new_df
new_df = grouped.apply(impute_height).reset_index(drop=True)

In [8]:
#create groups from the athlete data with the same Sex and NOC values
grouped = new_df.groupby(['Sex', 'NOC'])

# Create a function that imputes missing values with the mean Weight of the group, using the fillna method
def impute_weight(group):
    group['Weight'] = group['Weight'].fillna(group['Weight'].mean())
    return group

# Apply the function to each group and assign the results to new_df
new_df = grouped.apply(impute_weight).reset_index(drop=True)

### Format the Medals column

In [9]:
# Iterate over the DF and fill NaN medal values
for index, row in new_df.iterrows():
    if pd.isna(row['Medal']):
        new_df.at[index, 'Medal'] = 'None'

### Format the Names Column

In [10]:
#Initialize the new columns
new_df['First Name'] = ("")
new_df['Middle Name'] = ("")
new_df['Last Name'] = ("")
new_df = new_df.rename(columns={'Name': 'Full Name'})

#Re-Order new_df columns
new_df = new_df[['ID', 'Full Name', 'First Name', 'Middle Name', 'Last Name', 'Sex', 'Age', 'Height', 'Weight',
               'Team', 'NOC', 'Games', 'Year', 'Season', 'City', 'Sport', 'Event', 'Medal']]

#Split Full Name on spaces
split_names = new_df['Full Name'].str.split(' ', n=2, expand=True)

#Assign split names to new columns
new_df[['First Name', 'Middle Name', 'Last Name']] = split_names

#Handle Users without Middle Names
for index,row in new_df.iterrows():
    if row['Middle Name'] != None and row['Last Name'] == None:
        new_df.at[index, 'Last Name'] = row['Middle Name']
        new_df.at[index, 'Middle Name'] = 'None'
    else:
        pass


### Client Preference: Remove rows with duplicate Name, Team, Year, Event, and Medal Values

After investigating the duplicates removed for validity purposes, I noticed some users who had the exact same name, team, year, event, and medal values were in fact valid. Auguste Albert who competed under the same name, team, year, and event, recieved a bronze and silver medal in the same event. Along with medals, many other duplicates were being removed for having unknown events. Rows with duplicate Name, Team, Year, Event, and Medal values make up roughly 1300 rows, less than .5% of the data.

In [11]:
# create a boolean mask of rows with duplicate "PumpTagIndex" and "PumpStart" values
mask = new_df.duplicated(subset =['Full Name','Team', 'Year', 'Event', 'Medal'], keep = 'first')

#now let's drop the duplicate rows
new_df = new_df[~mask]

# reset the index after dropping rows
new_df.reset_index(drop=True, inplace=True)

## Export Cleaned File to CSV

In [12]:
new_df.to_csv("Olympic Athlete Data.csv")